# LLM Deployment Notebook (Colab-ready)
This notebook is self-contained for the Week 3 assignment. It uses open-source LLMs (Llama-style) and FAISS for RAG.
**Security note**: enter your ngrok token at runtime when prompted; do not hardcode secrets.

In [ ]:
!pip install -q transformers torch sentence-transformers faiss-cpu flask pyngrok

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from flask import Flask, request, jsonify
from pyngrok import ngrok

In [ ]:
# Cell 3: Knowledge base data (manual definition)

KNOWLEDGE_BASE = [
    {
        "id": "doc1",
        "title": "Shoplite User Registration Process",
                  "content": """Creating a seller account on Shoplite is a structured process designed to protect buyers 
        and maintain marketplace trust. Sellers must complete the seller registration process, which begins 
        by providing accurate business information such as legal business name, tax ID, business address, 
        and banking details. Shoplite performs business verification to ensure that each seller is a legitimate 
        entity and complies with local regulations, financial reporting, and tax obligations. Verification 
        typically takes 2-3 business days, during which Shoplite may request additional documents, such as 
        government-issued IDs, business licenses, or proof of tax registration. Sellers can track the status 
        of their verification at every step. Only after successful verification can a seller begin listing 
        products and accepting orders. The system is intentionally designed to avoid instant approval, 
        and personal accounts cannot be used for seller operations to ensure accountability. During onboarding, 
        step-by-step guidance is provided to ensure sellers understand their obligations, including compliance 
        with Shoplite’s seller policies, dispute resolution mechanisms, and marketplace standards. Completing 
        seller registration and verification ensures that buyers can trust the products offered, protects 
        financial transactions, and maintains the integrity of Shoplite’s platform. Sellers are encouraged 
        to carefully follow all instructions and provide complete documentation to avoid delays in the 2-3 
        business day verification process. This approach balances efficiency with security, enabling sellers 
        to operate confidently while preserving the marketplace ecosystem."""
    },
    {
        "id": "doc2",
        "title": "Shoplite Shopping Cart Features",
"content": """Shoplite’s return and refund system is designed to protect both buyers and sellers while 
        maintaining clarity and fairness. Most items are covered under a standard 30-day return window, which 
        begins from the date of delivery. Buyers seeking to return items must request a return authorization 
        through their order page and follow the seller’s instructions for preparing and shipping the return. 
        The system integrates order tracking through carrier APIs, ensuring that buyers can monitor the progress 
        of both shipments and returns. Sellers are expected to approve or reject return requests promptly; 
        disputes may be escalated to Shoplite arbitration if necessary. Once a return is authorized and the 
        item has been received and inspected, refunds are issued to the original payment method. Shoplite 
        emphasizes the importance of adhering to the 30-day return window; policies such as lifetime returns 
        are not supported. The combination of return authorization, structured seller instructions, and 
        automated order tracking ensures that the return process is transparent and reduces errors or disputes. 
        Buyers can also view return status and tracking information in their order history, creating a seamless 
        and predictable experience. Sellers are encouraged to communicate clearly, follow all platform guidelines, 
        and maintain high standards of item description and packaging to reduce returns and disputes."""
    },
    {
        "id": "doc3",
        "title": "Shoplite Checkout and Payment Flow",
"content": """Shoplite takes payment security seriously to protect buyers and sellers. All transactions 
        are processed using PCI-compliant payment processors, and sensitive payment information is tokenized 
        to prevent unauthorized access. Card numbers, bank account details, and other credentials are never 
        stored directly on Shoplite servers. Fraud prevention is implemented through multiple layers, including 
        device fingerprinting, velocity checks to monitor unusual transaction patterns, and risk scoring to 
        evaluate transaction legitimacy. The platform supports multiple payment methods, including credit cards, 
        PayPal, Apple Pay, and Google Pay, with seamless integration into the checkout flow. Multi-seller orders 
        are split programmatically, ensuring that each seller receives the correct portion of the payment, while 
        Shoplite retains its commission. These processes are carefully monitored and logged for audit purposes. 
        Machine learning algorithms detect potentially fraudulent activity in real-time, allowing Shoplite to 
        intervene before funds are released. Sellers are notified of high-risk orders or potential chargebacks. 
        By combining PCI-compliant systems, tokenization, and robust fraud prevention measures, Shoplite 
        ensures that payments are processed securely, efficiently, and with minimal risk to all parties 
        involved. This comprehensive approach protects both buyers’ financial information and sellers’ revenues, 
        while maintaining trust across the marketplace."""
    },
    {
        "id": "doc4",
        "title": "Shoplite Shipping and Delivery System",
"content": """Shoplite provides a shopping cart system that allows items to be saved and accessed across 
        multiple devices, ensuring a seamless shopping experience. Authenticated users have their carts persisted 
        across devices in real-time, allowing them to start shopping on a desktop, continue on a mobile device, 
        and finalize purchases on a tablet without losing items. For guest users, items are stored in cookies, 
        and when the user logs in, these items are merged at login with any existing cart contents. This ensures 
        that no selected item is lost and prevents duplication or loss of stock availability. The cart system 
        also includes features such as quantity adjustments, item removal, automated price recalculation with 
        discounts, tax estimation, and shipping cost calculations. Real-time synchronization with inventory 
        prevents checkout errors caused by out-of-stock items. Multi-seller support allows items from different 
        vendors to be combined seamlessly, maintaining an accurate cart state across all sessions. The system 
        is designed for reliability, user convenience, and to enhance conversion by reducing friction during 
        checkout. Cart persistence across devices increases engagement, provides a consistent experience, and 
        aligns inventory, payment state, and session management across all Shoplite platforms."""
    },
    {
        "id": "doc5",
        "title": "Shoplite Order Management",
"content": """Shoplite provides a standardized seller payout schedule to ensure transparency and 
        predictability for sellers. Payouts are typically made on a net-7 basis, meaning funds from completed 
        transactions are disbursed seven days after settlement. Sellers are required to complete business 
        verification before receiving payouts, which includes confirmation of banking details, tax ID, and 
        legal business registration. New sellers or those with pending verification may experience temporary 
        holds on payouts until compliance is confirmed. Multi-seller orders are processed to split payments 
        programmatically, ensuring each seller receives their accurate share while Shoplite retains the 
        appropriate commission. Sellers can monitor pending, completed, and scheduled payouts through the 
        vendor dashboard, providing full visibility into transaction flows. The payout process is designed to 
        balance timely access to funds with fraud prevention and operational integrity. By maintaining a 
        verification step and adhering to the net-7 schedule, Shoplite ensures that sellers are paid promptly 
        while minimizing risk. Clear communication, detailed reporting, and a structured payout workflow create 
        confidence for sellers and support long-term marketplace sustainability."""
    },
    {
        "id": "doc6",
        "title": "Shoplite Returns and Refunds Policy",
"content": """Shoplite has established a structured support flow to allow sellers and buyers 
        to report suspected fraudulent orders. When a potentially fraudulent order is identified, users 
        must provide detailed order information through the customer support interface. The submitted 
        case is triaged by Shoplite’s dedicated fraud team, which assesses the risk level using historical 
        data, machine-learning risk scoring, and rule-based checks. High-risk orders may be held for 
        verification before shipment or payment release. The fraud team works closely with payment 
        processors and internal monitoring systems to ensure that fraudulent activities are intercepted 
        efficiently. Sellers are notified about actions taken and may be asked to provide additional 
        documentation or evidence to aid in the investigation. Shoplite also tracks patterns of repeated 
        suspicious behavior, allowing proactive measures to prevent future fraud. The support flow ensures 
        that reports are handled systematically, transparently, and in compliance with PCI-DSS and 
        consumer protection regulations. Users are encouraged to report any abnormal transactions promptly, 
        and the fraud team prioritizes high-risk cases to minimize financial losses for both buyers and sellers. 
        By integrating a structured support flow, dedicated fraud team, and triaging mechanisms, Shoplite 
        maintains marketplace trust, mitigates risks, and ensures that legitimate transactions proceed safely."""
    },
    {
        "id": "doc7",
        "title": "Shoplite Product Listing Guidelines",
"content": """Shoplite provides REST API access for developers and enterprise clients, allowing 
        integration with inventory, orders, and payment reconciliation. API keys are required for authentication, 
        and each key is subject to rate limits to ensure system stability and fair usage across all clients. 
        Details of rate limits depend on the API key tier and can be found in the official API documentation. 
        Developers are strongly encouraged to implement exponential backoff strategies when requests exceed 
        thresholds, to reduce throttling and maintain operational continuity. The API supports endpoints for 
        product uploads, inventory updates, order retrieval, and payout tracking. SDKs for popular languages 
        including Python, Java, and JavaScript provide additional convenience for integrating with Shoplite’s 
        platform. Error handling, logging, and monitoring are recommended best practices to ensure reliable 
        API consumption. By enforcing rate limits, validating API keys, and using exponential backoff strategies, 
        Shoplite protects platform integrity while enabling developers to build scalable, automated solutions 
        efficiently and safely."""
    },
    {
        "id": "doc8",
        "title": "Shoplite Vendor Dashboard",
 "content": """Shoplite requires that returns are always authorized before shipment back to the seller. 
        Buyers must request return authorization through their order page, providing reasons and supporting 
        evidence such as photos for defective or misrepresented items. Once authorized, the item must be 
        shipped according to seller instructions. Upon receipt, the item undergoes inspection to verify 
        condition, completeness, and compliance with the return policy. Refunds are then issued to the original 
        payment method, and processing may take several business days depending on the payment provider. 
        Shoplite monitors all returns and refunds to ensure sellers comply with policies and maintain quality 
        standards. Unauthorized returns are rejected, and buyers are guided through the correct procedures. 
        The system integrates with order tracking and buyer notifications to maintain transparency throughout 
        the process. Returns and refunds management includes auditing logs, dispute resolution support, and 
        fraud detection to prevent misuse. By combining return authorization, inspection, and structured 
        refund issuance, Shoplite ensures buyers are protected while sellers can manage returns efficiently, 
        preserving trust across the marketplace ecosystem."""
    },
    {
        "id": "doc9",
        "title": "Shoplite Inventory Management",
"content": """Shoplite enables sellers to manage inventory programmatically through multiple 
        channels. The REST API allows real-time updates to product availability, pricing, and descriptions. 
        Sellers can also perform bulk updates using CSV uploads, which simplifies large-scale inventory 
        management for extensive catalogs. Webhooks provide notifications about changes in orders, stock 
        levels, or product status, allowing external systems to synchronize automatically. Integration with 
        inventory management software or ERP systems is supported, ensuring accurate, consistent data 
        across multiple sales channels. The platform validates data to prevent overselling or inconsistent 
        inventory states. Sellers are advised to monitor webhook logs and API responses for errors and 
        confirmation of successful updates. The combination of REST API, CSV uploads, and webhooks empowers 
        sellers to efficiently control stock, respond to market demand, and maintain accurate product 
        availability information for buyers. By leveraging these tools, Shoplite ensures operational efficiency, 
        minimizes order errors, and supports scalable growth for sellers of all sizes."""
    },
    {
        "id": "doc10",
        "title": "Shoplite Buyer Protection Program",
"content": """Most items sold on Shoplite fall under a standard 30-day return window. This period 
        starts from the date of delivery to the buyer and applies to new, unused, and properly packaged items. 
        Sellers may specify exceptions, which are clearly communicated in product listings. During this window, 
        buyers can request return authorization, follow instructions for returning items, and receive refunds 
        after inspection. The 30-day return window balances buyer protection with operational efficiency for 
        sellers, reducing long-term disputes and maintaining marketplace trust. Shoplite integrates tracking 
        systems and notifications to guide buyers through the return process, ensuring transparency and 
        accountability. By standardizing the 30-day return window, Shoplite establishes predictable rules 
        for returns and refunds while allowing sellers to manage exceptions responsibly."""
    },
    {
        "id": "doc11",
        "title": "Shoplite Vendor Onboarding Program",
"content": """Shoplite users can enhance account security by enabling two-factor authentication (2FA). 
        2FA can be activated from the account settings page, where users select their preferred verification 
        method. Options include SMS-based one-time codes or authentication apps that generate time-based codes. 
        After setup, every login requires the user to provide both their password and the second factor, ensuring 
        protection even if the password is compromised. Recovery methods, such as backup codes, are also 
        provided to prevent account lockout. Enabling 2FA strengthens account security, safeguards buyer 
        information, and reduces the risk of unauthorized access to sensitive seller and buyer data."""
    },
    {
        "id": "doc12",
        "title": "Shoplite Customer Support System",
"content": """Shoplite provides sellers with flexible shipping configuration options. Sellers can 
        define zone-based rates depending on geographic regions, set flat rates for specific shipping methods, 
        or integrate calculated carrier rates using real-time logistics APIs. Both merchant-fulfilled and 
        third-party logistics (3PL) options are supported, allowing sellers to select their preferred fulfillment 
        strategy. These configurations ensure accurate cost calculation, delivery estimation, and compatibility 
        with Shoplite’s order tracking system. Sellers are encouraged to maintain consistency and transparency 
        in shipping policies to reduce disputes and enhance buyer satisfaction."""
    },
    {
        "id": "doc13",
        "title": "Shoplite Fraud Detection and Security",
  "content": """Shoplite enforces review moderation to maintain credibility. Automated filters scan all 
        submissions for spam, offensive content, or suspicious patterns. Flagged reviews are escalated for 
        manual review by trained moderators. Verified purchase badges are applied to reviews from customers 
        who have completed a genuine transaction. This system ensures authenticity, prevents abuse, and 
        maintains trust for both buyers and sellers. Sellers are notified when reviews are removed or 
        flagged, providing transparency and feedback on product perception."""
    },
    {
        "id": "doc14",
        "title": "Shoplite Mobile App Features",
"content": """Shoplite supports multiple locales and currencies to serve international buyers and 
        sellers. Content such as product descriptions, currency symbols, and interface text is localized 
        for major markets. Sellers can configure listings with country-specific prices, shipping rules, 
        and translated content. Buyers experience a seamless interface in their preferred language and 
        currency, while sellers maintain consistent product management globally. Localization improves 
        conversion rates, accessibility, and overall marketplace usability."""
    },
    {
        "id": "doc15",
        "title": "Shoplite Advertising Solutions",
 "content": """Shoplite maintains audit logs for all administrative actions, including user management, 
        payment adjustments, and content changes. By default, logs are retained for 90 days, though legal 
        requirements may mandate longer retention in certain jurisdictions. Audit logs capture timestamps, 
        user IDs, and action details, providing accountability, traceability, and support for dispute resolution. 
        Administrators and security teams can review logs for compliance, operational oversight, and forensic 
        investigations."""
    },
    {
        "id": "doc16",
        "title": "Shoplite Loyalty and Rewards Program",
"content": """Shoplite provides tools to manage chargebacks effectively. When a buyer disputes a 
        transaction, sellers can collect evidence such as invoices, shipment tracking, and communications 
        to support their case. Shoplite mediates disputes and may hold funds until resolution is achieved. 
        Chargeback management ensures fair handling of contested payments, protects sellers from unjust 
        loss, and maintains overall marketplace integrity."""
    },
    {
        "id": "doc17",
        "title": "Shoplite Analytics and Reporting",
 "content": """Shoplite enforces coupon stacking and exclusion rules at checkout. Merchants define 
        how multiple promotional codes can be applied, and the system validates these rules automatically. 
        This prevents misuse, ensures correct discount calculation, and allows sellers to maintain desired 
        pricing strategies. Coupon stacking rules provide flexibility while preserving platform integrity 
        and buyer confidence."""
    },
    {
        "id": "doc18",
        "title": "Shoplite API Integration",
 "content": """If an order is delayed by a carrier, Shoplite triggers support workflows to manage the 
        exception. Customers can request updates or refunds depending on the seller’s policy. Real-time 
        tracking and notifications inform both buyers and sellers about status changes. This structured 
        response ensures transparency, minimizes frustration, and maintains buyer trust in the marketplace."""
    },
    {
        "id": "doc19",
        "title": "Shoplite Compliance and Legal Policies",
   "content": """Developers can obtain API keys by registering an account and requesting access 
        through the developer portal. Keys must be stored securely and are associated with rate limits 
        and permissions. API keys enable programmatic access to inventory, orders, and payments. 
        Proper management ensures integration security and platform compliance."""
    },
    {
        "id": "doc20",
        "title": "Shoplite Future Roadmap",
    "content": """Shoplite uses a combination of rule-based checks and machine-learning risk scoring 
        to prevent fraudulent transactions. These systems integrate with external identity sources and 
        monitor patterns in purchasing, payment, and account activity. Alerts, holds, and additional 
        verification steps are applied based on risk assessments, ensuring that both buyers and sellers 
        are protected from fraud while maintaining smooth marketplace operations."""
    }
]

print(f"Loaded {len(KNOWLEDGE_BASE)} KB docs (manually defined)")


In [ ]:
# ===============================
# Cell 4: Prompts (direct Python dicts)
# ===============================
PROMPTS = {
    "base_retrieval_prompt": {
        "role": "You are a helpful Shoplite customer service assistant.",
        "goal": "Provide accurate answers using only the provided Shoplite documentation.",
        "context_guidelines": [
            "Use only information from the provided document snippets",
            "Cite specific documents when possible"
        ],
        "response_format": "Answer: [Your response based on context]\nSources: [List document titles referenced]"
    },
    "complex_multi_doc_prompt": {
        "role": "You are a Shoplite knowledge synthesis assistant.",
        "goal": "Synthesize information from multiple documents to answer complex questions, and list supporting documents.",
        "context_guidelines": [
            "If multiple documents provide partial answers, combine them and cite each source",
            "If no sufficient context is found, say so and ask for clarification"
        ],
        "response_format": "Answer: [Synthesis]\nSources: [Doc titles]"
    },
    "refusal_and_clarification_prompt": {
        "role": "You are a safe Shoplite assistant that refuses to answer when outside scope.",
        "goal": "Refuse to answer when the knowledge base lacks relevant context or when the request involves PII or policy violations.",
        "context_guidelines": [
            "If requested data is not in the knowledge base, respond with a refusal and a short explanation",
            "When user query is ambiguous, ask one clarifying question"
        ],
        "response_format": "Answer: [Refusal or Clarification]\nReason: [Short rationale]"
    },
    "short_customer_facing_snippet": {
        "role": "You are a concise Shoplite customer assistant.",
        "goal": "Provide short, actionable answers suitable for UI tooltips or quick replies.",
        "context_guidelines": [
            "Limit answers to 2-3 sentences",
            "Include a single actionable next step when possible"
        ],
        "response_format": "Answer: [2-3 sentence reply]\nActions: [Optional next step]"
    }
}

print("Available prompts:", list(PROMPTS.keys()))


In [ ]:
# ===============================
# Cell 5: Load Open Source LLM
# (FLAN-T5 Large – no Hugging Face token required)
# ===============================

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = "google/flan-t5-large"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

llm_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512
)

print("✅ FLAN-T5 Large loaded successfully")


In [ ]:
# ===============================
# Cell 6: Embeddings + FAISS Index
# ===============================
embedder = SentenceTransformer("all-MiniLM-L6-v2")

corpus = [doc["content"] for doc in KNOWLEDGE_BASE]
doc_embeddings = embedder.encode(corpus, convert_to_numpy=True, show_progress_bar=True)

dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

def retrieve_docs(query, top_k=3):
    query_embedding = embedder.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k)
    results = [KNOWLEDGE_BASE[i] for i in indices[0]]
    return results

In [ ]:
# ===============================
# Cell 7: Response Generation
# ===============================
def generate_response(query, prompt_type="base_retrieval_prompt"):
    retrieved = retrieve_docs(query)
    context_text = "\n\n".join([f"{doc['title']}: {doc['content']}" for doc in retrieved])

    prompt_cfg = PROMPTS[prompt_type]
    system_prompt = f"""{prompt_cfg['role']}
Goal: {prompt_cfg['goal']}

Context Guidelines:
- {'; '.join(prompt_cfg['context_guidelines'])}

User Query: {query}

Context:
{context_text}

{prompt_cfg['response_format']}
"""

    response = llm_pipeline(system_prompt, max_new_tokens=300, temperature=0.2)[0]["generated_text"]
    return {
        "answer": response,
        "sources": [doc["title"] for doc in retrieved]
    }

In [ ]:
# ===============================
# Cell 8: Flask API
# ===============================
app = Flask(__name__)

@app.route("/chat", methods=["POST"])
def chat():
    data = request.get_json()
    query = data.get("query", "")
    prompt_type = data.get("prompt_type", "base_retrieval_prompt")
    result = generate_response(query, prompt_type)
    return jsonify(result)

@app.route("/ping", methods=["GET"])
def ping():
    return jsonify({"status": "LLM alive!"})

@app.route("/health", methods=["GET"])
def health():
    return jsonify({"status": "ok", "docs_loaded": len(KNOWLEDGE_BASE)})

In [ ]:
# ===============================
# Cell 9: ngrok Tunnel Setup (Non-Blocking Flask)
# ===============================
from threading import Thread
import time
import requests

# Ask for ngrok token
ngrok_token = input("Enter your ngrok token: ")
ngrok.set_auth_token(ngrok_token)

# Start ngrok tunnel
public_url = ngrok.connect(5000).public_url
print(f"✅ Public URL: {public_url}")

# Function to run Flask app
def run_app():
    app.run(port=5000, use_reloader=False)

# Run Flask in background thread
thread = Thread(target=run_app, daemon=True)
thread.start()

# Wait a bit for server to boot
time.sleep(2)

# Quick health check
try:
    resp = requests.get(f"{public_url}/health")
    if resp.status_code == 200:
        print("✅ Flask server is running correctly.")
        print("Health check response:", resp.json())
    else:
        print(f"⚠️ Server started but health check failed. Status code: {resp.status_code}")
except Exception as e:
    print("❌ Could not reach server:", e)


In [ ]:
# ===============================
# Cell 10: Evaluation on 20 Ground-Truth Questions (Updated)
# ===============================

# Ground-truth questions with required and forbidden keywords
GROUND_TRUTH_QA = [
    {
        "q": "How do I create a seller account on Shoplite?",
        "required": ["seller registration", "business verification", "2-3 business days"],
        "forbidden": ["instant approval", "no verification required", "personal accounts"]
    },
    {
        "q": "What are Shoplite's return policies and how do I track my order status?",
        "required": ["30-day return window", "order tracking", "return authorization"],
        "forbidden": ["no returns accepted", "lifetime returns"]
    },
    {
        "q": "How does Shoplite handle payment security?",
        "required": ["PCI-compliant", "tokenized", "fraud prevention"],
        "forbidden": []
    },
    {
        "q": "Can I save items in my cart across devices?",
        "required": ["persisted across devices", "cookies", "merged at login"],
        "forbidden": []
    },
    {
        "q": "What is the standard seller payout schedule?",
        "required": ["net-7", "payouts", "verification"],
        "forbidden": []
    },
    {
        "q": "How do I report a fraudulent order?",
        "required": ["support flow", "fraud team", "triaged"],
        "forbidden": []
    },
    {
        "q": "What are Shoplite's API rate limits?",
        "required": ["rate limits", "API keys", "exponential backoff"],
        "forbidden": []
    },
    {
        "q": "How are returns processed and refunds issued?",
        "required": ["return authorization", "refunds", "inspection"],
        "forbidden": []
    },
    {
        "q": "How can sellers manage inventory programmatically?",
        "required": ["REST API", "CSV uploads", "webhooks"],
        "forbidden": []
    },
    {
        "q": "What is the standard return window on Shoplite?",
        "required": ["30-day return window"],
        "forbidden": []
    },
    {
        "q": "How do I enable two-factor authentication (2FA)?",
        "required": ["account settings", "2FA", "authenticator app"],
        "forbidden": []
    },
    {
        "q": "What shipping options can sellers configure?",
        "required": ["zone-based", "flat rates", "carrier rates"],
        "forbidden": []
    },
    {
        "q": "How are reviews moderated?",
        "required": ["automated filters", "manual review", "verified purchase"],
        "forbidden": []
    },
    {
        "q": "What localization features does Shoplite support?",
        "required": ["locales", "currencies", "localized content"],
        "forbidden": []
    },
    {
        "q": "What data is retained in audit logs?",
        "required": ["audit logs", "90 days"],
        "forbidden": []
    },
    {
        "q": "How does Shoplite handle chargebacks?",
        "required": ["chargebacks", "evidence collection"],
        "forbidden": []
    },
    {
        "q": "What are the rules for coupon stacking?",
        "required": ["coupon stacking", "exclusion rules"],
        "forbidden": []
    },
    {
        "q": "What happens if an order is delayed by the carrier?",
        "required": ["delivery exceptions", "support workflows"],
        "forbidden": []
    },
    {
        "q": "How do I get developer API keys?",
        "required": ["developer portal", "API keys"],
        "forbidden": []
    },
    {
        "q": "What measures are in place for fraud prevention?",
        "required": ["rule-based checks", "machine-learning risk scoring"],
        "forbidden": []
    }
]

# Function to dynamically detect prompt type
def detect_prompt_type(question):
    """
    Detects the most suitable prompt type based on question content.
    """
    question_lower = question.lower()
    
    # Short/concise answers
    if any(keyword in question_lower for keyword in ["quick", "tip", "short", "ui", "tooltip"]):
        return "short_customer_facing_snippet"
    
    # Refusal/clarification needed
    if any(keyword in question_lower for keyword in ["personal info", "password", "pii", "forbidden"]):
        return "refusal_and_clarification_prompt"
    
    # Complex multi-document synthesis
    if any(keyword in question_lower for keyword in ["return", "order delayed", "track", "fraudulent", "report", "policies"]):
        return "complex_multi_doc_prompt"
    
    # Default to base retrieval
    return "base_retrieval_prompt"

# Evaluation loop
total_pass = 0

for i, item in enumerate(GROUND_TRUTH_QA, 1):
    print(f"\n==================== Q{i:02d} ====================")
    
    # Detect prompt type dynamically
    prompt_type = detect_prompt_type(item["q"])
    
    # Generate response using your function
    result = generate_response(item["q"], prompt_type=prompt_type)
    answer = result['answer']
    sources = result['sources']

    print(f"Prompt Type: {prompt_type}")
    print(f"Question: {item['q']}")
    print(f"Answer:\n{answer}")
    print(f"Sources: {sources}")

    # Check required keywords
    missing_required = [kw for kw in item["required"] if kw.lower() not in answer.lower()]
    forbidden_present = [fw for fw in item["forbidden"] if fw.lower() in answer.lower()]

    passed = not missing_required and not forbidden_present
    if passed:
        total_pass += 1

    print(f"✅ Required keywords present? {'Yes' if not missing_required else 'No -> Missing: ' + ', '.join(missing_required)}")
    if forbidden_present:
        print(f"❌ Forbidden content found: {', '.join(forbidden_present)}")
    else:
        print("✅ No forbidden content detected")

print(f"\nFinal Score: {total_pass}/{len(GROUND_TRUTH_QA)} passed")
